# Localización poblaciones

## Introducción

El siguiente objetivo al que nos vamos a enfocar es a localizar las poblaciones más importantes en cada foto. Vamos a utilizar por ejemplo el criterio de que superen los 100.000 habitantes, por lo que tendremos que encontrar alguna fuente de datos que nos permita buscar en un rango de coordenadas (las que caigan dentro de la fotografía o idealmente de la porción visible circular de la misma) y que contenga también el número de habitantes para que podamos filtrar.

## Servicios de geolocalización

Probamos con todos los [servicios de geolocalización soportados dentro de la librería GeoPy](https://geopy.readthedocs.io/en/latest/#module-geopy.geocoders), pero ninguno nos ofreció la funcionalidad que buscábamos. Finalmente encontramos una [base de datos estática con todas las poblaciones del mundo con más de 15.000 habitantes](https://datacore-gn.unepgrid.ch/geonetwork//srv/spa/catalog.search#/metadata/4a64faed-8674-4bb2-baad-fb6446ee3a6d) que contiene 26420 entradas. En concreto el fichero utilizado ha sido [éste](https://download.geonames.org/export/dump/cities15000.zip) cuyo contenido se describe en [este documento](https://download.geonames.org/export/dump/readme.txt).

## Manejo de la base de datos de ciudades

El fichero `cities15000.txt` podrá leerse con la función [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) de la librería Pandas. Tendremos que incorporar la primera fila que sirve para identificar las columnas; al nuevo fichero le damos el nombre `cities15000_with_header.txt`. Luego a la hora de cargarlo tendremos que indicar que el separador de las columnas es el caracter tabulador.

En el siguiente bloque de código vamos a hacer una prueba de manejo del fichero con Pandas. Haremos una búsqueda de las ciudades que tengan como nombre `Zaragoza` (un pequeño tutorial sobre cómo hacer filtrados sobre los DataFrames de Pandas puede encontrarse [aquí](https://www.geeksforgeeks.org/ways-to-filter-pandas-dataframe-by-column-values/)).

In [14]:
import os
import pandas

CITIES_DIR_PATH = "db"
CITIES_FILE = "cities15000_with_header.txt"

path = os.path.dirname(os.path.realpath("__file__"))
cities_dir_path = os.path.join(path, CITIES_DIR_PATH)
cities_file_path = os.path.join(cities_dir_path, CITIES_FILE)

cities_data = pandas.read_csv(cities_file_path, sep='\t')

cities_data[cities_data['name'] == 'Zaragoza']

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
5295,3665566,Zaragoza,Zaragoza,Zaragoza,7.48971,-74.86919,P,PPLA2,CO,NaN,02,05895,NaN,NaN,24067,NaN,66,America/Bogota,2022-08-26
7863,3104324,Zaragoza,Zaragoza,"Caesaraugusta,Caesarea Augusta,Caragoca,Saldub...",41.65606,-0.87734,P,PPLA,ES,NaN,52,Z,50297,NaN,675301,NaN,214,Europe/Madrid,2022-10-22
10100,3587543,Zaragoza,Zaragoza,Zaragoza,14.64968,-90.89034,P,PPLA2,GT,NaN,03,415,NaN,NaN,24022,NaN,2062,America/Guatemala,2023-02-12


## Ciudades que aparecen en una foto

Ahora que sabemos manejar la base de datos de ciudades, vamos a intentar obtener el listado de ciudades que aparecen teóricamente dentro de una foto determinada. En el bloque de código que vamos a escribir para desarrollar esta idea de nuevo utilizaremos la foto 493 como ejemplo en la que sabemos que como mínimo debería aparecer la ciudad de Pamplona. Filtraremos la base de datos por las poblaciones de más de 100.000 habitantes, cuyas coordenadas latitud/longitud estén dentro del rango cubierto por la foto.

In [23]:
import os
import pandas
from geopy import distance, point

CITIES_DIR_PATH = "db"
CITIES_FILE = "cities15000_with_header.txt"
FOTO_W = 3280
FOTO_H = 2464
S_W = 6.287
F = 5
MIN_POP = 100000

# Datos Foto 493
pnt_I = point.Point(43.097612, -2.394949)
alpha = -34.16
alt = 420981

path = os.path.dirname(os.path.realpath("__file__"))
cities_dir_path = os.path.join(path, CITIES_DIR_PATH)
cities_file_path = os.path.join(cities_dir_path, CITIES_FILE)

cities_data = pandas.read_csv(cities_file_path, sep='\t')

# Corregimos el punto central de la foto
pnt_O = distance.great_circle(meters=52329).destination(pnt_I, bearing=-61.69-alpha)
pnt_O.altitude = alt

# Calculamos el rango de latitudes y longitudes mostradas por la foto
d_w = S_W * pnt_O.altitude / F
d_h = d_w * FOTO_H / FOTO_W
pnt_T = distance.great_circle(meters=d_h/2).destination(pnt_O, bearing=0)
pnt_B = distance.great_circle(meters=d_h/2).destination(pnt_O, bearing=180)
pnt_L = distance.great_circle(meters=d_w/2).destination(pnt_O, bearing=270)
pnt_R = distance.great_circle(meters=d_w/2).destination(pnt_O, bearing=90)
t_lat, t_lon = pnt_T.latitude, pnt_T.longitude
b_lat, b_lon = pnt_B.latitude, pnt_B.longitude
l_lat, l_lon = pnt_L.latitude, pnt_L.longitude
r_lat, r_lon = pnt_R.latitude, pnt_R.longitude

# Hacemos la consulta
cities_data[(cities_data['latitude'] <= t_lat) &
            (cities_data['latitude'] >= b_lat) &
            (cities_data['longitude'] <= r_lon) &
            (cities_data['longitude'] >= l_lon) &
            (cities_data['population'] >= MIN_POP)]

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
7866,3104499,Gasteiz / Vitoria,Gasteiz / Vitoria,"Bittorixa,Gasteiz,VIT,Victoriacum,Vitoria,Vito...",42.84998,-2.67268,P,PPLA,ES,NaN,59,VI,01059,NaN,249176,NaN,547,Europe/Madrid,2022-10-22
7918,3109718,Santander,Santander,"Portus Victoriae Iuliobrigensium,SDR,Sanandere...",43.46472,-3.80444,P,PPLA,ES,NaN,39,S,39075,NaN,172044,NaN,28,Europe/Madrid,2022-10-22
7923,3110044,Donostia / San Sebastián,San Sebastian,"Donosti,Donostia,Donostia / San Sebastian,Dono...",43.31283,-1.97499,P,PPLA2,ES,NaN,59,SS,20069,NaN,185357,NaN,17,Europe/Madrid,2020-03-18
7969,3114472,Pamplona,Pamplona,"Iruinea,Iruna,Irunea,Iruña,Iruñea,Lungsod ng I...",42.81687,-1.64323,P,PPLA,ES,NaN,32,NaN,31201,NaN,209672,460.0,455,Europe/Madrid,2022-10-22
7973,3114711,Oviedo,Oviedo,"Aueda,OVI,Ov'edo,Ovedo,Ovetum,Oviedo,Oviedu,Ov...",43.36029,-5.84476,P,PPLA,ES,NaN,34,O,33044,NaN,220020,NaN,237,Europe/Madrid,2022-10-22
8007,3118150,Logroño,Logrono,"Lagron'ja,Logron'jo,Logron'o,Logronh,Logronio,...",42.46667,-2.45000,P,PPLA,ES,NaN,27,LO,26089,NaN,152485,NaN,389,Europe/Madrid,2022-10-22
8011,3118532,León,Leon,"Ciuda de Llion,Ciudá de Llión,LEN,Leon,Leono,L...",42.60003,-5.57032,P,PPLA2,ES,NaN,55,LE,24089,NaN,134305,NaN,844,Europe/Madrid,2013-07-06
8037,3121424,Gijón,Gijon,"Chichonas,Gigia,Gijon,Gijon/Xixon,Gijón,Gijón/...",43.53573,-5.66152,P,PPLA3,ES,NaN,34,O,33024,NaN,277554,NaN,22,Europe/Madrid,2019-09-05
8086,3127461,Burgos,Burgos,"Bourgos,Burgas,Burgi,Burgos,Burgosa,Burgosas,B...",42.34106,-3.70184,P,PPLA2,ES,NaN,55,BU,09059,NaN,178966,NaN,863,Europe/Madrid,2019-09-05
8090,3128026,Bilbao,Bilbao,"BIO,Bil'baa,Bil'bao,Bilbao,Bilbau,Bilbaum,Bilb...",43.26271,-2.92528,P,PPLA2,ES,NaN,59,BI,48020,NaN,345821,NaN,20,Europe/Madrid,2022-09-23


Vemos que en el listado salen ciudades que claramente no están en la foto (por ejemplo Gijón). Esto ocurre por el problema del ojo de buey en el que se colocó la AstroPi, que crea el marco circular que oculta la periferia de la foto. Más tarde cuando representemos la posición de las ciudades en la foto, veremos como algunas de ellas caen sobre ese marco oscuro circular.

## Señalización de ciudades en fotos

Ahora que tenemos la lista de ciudades contenidas en la foto 493, vamos a representarlas sobre ella. Sólo tenemos que reunir en un bloque de código algunas de las técnicas desarrolladas anteriormente. Para aproximarnos hacia la solución final automatizada, utilizaremos el fichero `atlantes_2021-2022.csv` para obtener la latitud/longitud/altitud de la foto y el fichero `alphas_445-622.json` para obtener el ángulo α.

In [18]:
import os
import cv2
import json
import pandas
from geopy import distance, point

# Constantes
PHOTO_W = 640
PHOTO_H = 480
S_W = 6.287
F = 5
PHOTO_ROT_DIR_PATH = "atlantes_2021-2022_rotadas"
ATLANTES_FILE = "atlantes_2021-2022.csv"
ALPHAS_FILE = "alphas_445-622.json"
LOGO_FILE = "logo_v1_20px.png"
PHOTO_START = 445#493
PHOTO_END = 622#496
MIN_POP = 150000
ANIM_FILE = "anim_%d-%d_%d.avi" % (PHOTO_START, PHOTO_END, MIN_POP)
FPS = 3
OUTPUT = True    # True: Salida a fichero; False: Salida a pantalla
MARKER = True     # True: Ciudades marcadas con marker; False: Marcadas con logo Atlantes
CITIES_DIR_PATH = "db"
CITIES_FILE = "cities15000_with_header.txt"


# Calcula el pixel (X,Y) correspondiente a unas coordenadas (latitud,longitud) de una foto cuyo pixel central tiene
# unas coordenadas conocidas.
# Argumentos:
#     * pnt_I: Punto (clase geopy.point.Point) medido por AstroPi durante la captura de la foto como posición
#              de la ISS.
#     * alt:   Altitud de la ISS durante la captura de la foto.
#     * alpha: Ángulo de la órbita respecto del ecuador durante la captura de la foto.
#     * pnt_X: Punto (clase geopy.point.Point) del que queremos obtener su posición en la foto.
# Resutado:
#     Tupla (X,Y) con la posición del pixel correspondiente a las coordenadas del punto pnt_X.
def coords2pixel(pnt_I, alpha, alt, pnt_X):
    pnt_O = distance.great_circle(meters=52329).destination(pnt_I, bearing=-61.69-alpha)
    pnt_O.altitude = alt
    
    d_w = S_W * pnt_O.altitude / F
    d_h = d_w * PHOTO_H / PHOTO_W
    pnt_T = distance.great_circle(meters=d_h/2).destination(pnt_O, bearing=0)
    pnt_B = distance.great_circle(meters=d_h/2).destination(pnt_O, bearing=180)
    pnt_L = distance.great_circle(meters=d_w/2).destination(pnt_O, bearing=270)
    pnt_R = distance.great_circle(meters=d_w/2).destination(pnt_O, bearing=90)
    t_lat, t_lon = pnt_T.latitude, pnt_T.longitude
    b_lat, b_lon = pnt_B.latitude, pnt_B.longitude
    l_lat, l_lon = pnt_L.latitude, pnt_L.longitude
    r_lat, r_lon = pnt_R.latitude, pnt_R.longitude
    x_lat, x_lon = pnt_X.latitude, pnt_X.longitude
    
    # Normalizamos las longitudes para que se puedan hacer los cálculos proporcionales
    if l_lon * r_lon < 0:
        if l_lon < -90:
            l_lon += 360
        if r_lon < -90:
            r_lon += 360
        if x_lon < -90:
            x_lon += 360
    
    if not (b_lat <= x_lat <= t_lat) or not (l_lon <= x_lon <= r_lon):
        raise Exception(f"Las coordenadas ({pnt_X.latitude},{x_lon}) quedan fuera de la foto.")

    X = PHOTO_W * ( 1 - (r_lon - x_lon) / (r_lon - l_lon))
    Y = PHOTO_H * (t_lat - x_lat) / (t_lat - b_lat)
    return (round(X), round(Y))


# Calcula las ciudades que caen dentro de una foto cuyo pixel central tiene unas coordenadas conocidas.
# Argumentos:
#     * pnt_I: Punto (clase geopy.point.Point) medido por AstroPi durante la captura de la foto como posición
#              de la ISS.
#     * alt:   Altitud de la ISS durante la captura de la foto.
#     * alpha: Ángulo de la órbita respecto del ecuador durante la captura de la foto.
# Resutado:
#     Pandas DataFrame con las ciudades que entran en la foto
def find_cities(pnt_I, alpha, alt):
    # Corregimos el punto central de la foto
    pnt_O = distance.great_circle(meters=52329).destination(pnt_I, bearing=-61.69-alpha)
    pnt_O.altitude = alt

    # Calculamos el rango de latitudes y longitudes mostradas por la foto
    d_w = S_W * pnt_O.altitude / F
    d_h = d_w * PHOTO_H / PHOTO_W
    pnt_T = distance.great_circle(meters=d_h/2).destination(pnt_O, bearing=0)
    pnt_B = distance.great_circle(meters=d_h/2).destination(pnt_O, bearing=180)
    pnt_L = distance.great_circle(meters=d_w/2).destination(pnt_O, bearing=270)
    pnt_R = distance.great_circle(meters=d_w/2).destination(pnt_O, bearing=90)
    t_lat, t_lon = pnt_T.latitude, pnt_T.longitude
    b_lat, b_lon = pnt_B.latitude, pnt_B.longitude
    l_lat, l_lon = pnt_L.latitude, pnt_L.longitude
    r_lat, r_lon = pnt_R.latitude, pnt_R.longitude

    # Hacemos la consulta
    return cities_data[(cities_data['latitude'] <= t_lat) &
                       (cities_data['latitude'] >= b_lat) &
                       (cities_data['longitude'] <= r_lon) &
                       (cities_data['longitude'] >= l_lon) &
                       (cities_data['population'] >= MIN_POP)]


# https://stackoverflow.com/questions/70578600/how-to-merge-one-rgba-and-one-rgb-images-in-opencv
def alpha_merge(small_foreground, background, x, y):
    """
    Puts a small BGRA picture in front of a larger BGR background.
    :param small_foreground: The overlay image. Must have 4 channels.
    :param background: The background. Must have 3 channels.
    :param x: X position where to put the overlay's center.
    :param y: Y position where to put the overlay's center.
    :return: a copy of the background with the overlay added.
    """
    result = background.copy()
    # From everything I read so far, it seems we need the alpha channel separately
    # so let's split the overlay image into its individual channels
    fg_b, fg_g, fg_r, fg_a = cv2.split(small_foreground)
    # Make the range 0...1 instead of 0...255
    fg_a = fg_a / 255.0
    # Multiply the RGB channels with the alpha channel
    label_rgb = cv2.merge([fg_b * fg_a, fg_g * fg_a, fg_r * fg_a])
    # Work on a part of the background only
    height, width = small_foreground.shape[0], small_foreground.shape[1]
    mid_h = int(height/2)
    mid_w = int(width/2)
    part_of_bg = result[y-mid_h:y+mid_h, x-mid_w:x+mid_w, :]
    # Same procedure as before: split the individual channels
    bg_b, bg_g, bg_r = cv2.split(part_of_bg)
    # Merge them back with opposite of the alpha channel
    part_of_bg = cv2.merge([bg_b * (1 - fg_a), bg_g * (1 - fg_a), bg_r * (1 - fg_a)])
    # Add the label and the part of the background
    cv2.add(label_rgb, part_of_bg, part_of_bg)
    # Replace a part of the background
    result[y-mid_h:y+mid_h, x-mid_w:x+mid_w, :] = part_of_bg
    return result


path = os.path.dirname(os.path.realpath("__file__"))
photo_rot_dir_path = os.path.join(path, PHOTO_ROT_DIR_PATH)
atlantes_file_path = os.path.join(path, ATLANTES_FILE)
alphas_file_path = os.path.join(path, ALPHAS_FILE)
cities_dir_path = os.path.join(path, CITIES_DIR_PATH)
cities_file_path = os.path.join(cities_dir_path, CITIES_FILE)

alphas_file = open(alphas_file_path)
alphas_json = json.load(alphas_file)

if not MARKER:
    logo_file_path = os.path.join(path, LOGO_FILE)
    logo_cv = cv2.imread(logo_file_path, cv2.IMREAD_UNCHANGED)

atlantes_data = pandas.read_csv(atlantes_file_path)
cities_data = pandas.read_csv(cities_file_path, sep='\t')

if OUTPUT:
    fourcc = cv2.VideoWriter_fourcc(*'MP42')
    anim_file_path = os.path.join(path, ANIM_FILE)
    video = cv2.VideoWriter(anim_file_path, fourcc, float(FPS), (PHOTO_W, PHOTO_H))

for i in range(PHOTO_START, PHOTO_END+1):
    pnt_I = point.Point(atlantes_data.latitude[i-1], atlantes_data.longitude[i-1])
    alpha = alphas_json[str(i)]
    altitude = atlantes_data.elevation[i-1]
    
    photo_path = os.path.join(photo_rot_dir_path, "atlantes_%d.jpg" % (i))
    photo_cv = cv2.imread(photo_path)
    
    photo_cv = cv2.resize(photo_cv, (PHOTO_W, PHOTO_H))
    
    cities = find_cities(pnt_I, alpha, altitude)
    for index, row in cities.iterrows():
        try:
            pnt_X = point.Point(row["latitude"], row["longitude"])
            (X, Y) = coords2pixel(pnt_I, alpha, altitude, pnt_X)
            if MARKER:
                # https://docs.opencv.org/4.1.2/d6/d6e/group__imgproc__draw.html
                #photo_cv = cv2.drawMarker(photo_cv, (X, Y), (255, 0, 0), markerType=cv2.MARKER_CROSS, markerSize=60, thickness=9)
                #photo_cv = cv2.circle(photo_cv, (X, Y), 20, (255, 0, 0), thickness=9)
                photo_cv = cv2.arrowedLine(photo_cv, (X+10, Y-10), (X, Y), (255, 0, 0), thickness=2, tipLength=0.5)
            else:
                photo_cv = alpha_merge(logo_cv, photo_cv, X, Y)
            photo_cv = cv2.putText(photo_cv, "%s (%s)" % (row["asciiname"], row["country code"]),
                                   (X+10, Y-12), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        except:
            pass
    
    if OUTPUT:
        video.write(photo_cv)
    else:
        cv2.imshow("%d-%d" % (PHOTO_START, PHOTO_END), photo_cv)
        cv2.waitKey(int(1000/FPS))

cv2.destroyAllWindows()

alphas_file.close()
if OUTPUT:
    video.release()